<a href="https://colab.research.google.com/github/DCDPUAEM/DCDP/blob/main/04%20Deep%20Learning/notebooks/06-CNN-II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Clasificación con Redes Neuronales Convolucionales</h1>

<h3>Parte II: Generadores, Embeddings y Modelos Pre-entrenados

En esta notebook usaremos una red neuronal convolucional (CNN) para clasificar el dataset *cats vs dogs* de kaggle. Observaremos, además, el efecto del dropout y analizaremos la información de las capas ocultas para ganar intuición sobre el funcionamiento interno de este tipo de redes.

Además, usaremos modelos pre-entrenados y con estos obtendremos embeddings para clasificar las imágenes. Para esto, usaremos la clase `Model` de Keras.

___

Verifiquemos que el entorno de ejecución en Colab sea GPU

In [ ]:
import tensorflow as tf

print('GPU presente en: {}'.format(tf.test.gpu_device_name()))

# [El dataset Dogs vs. Cats](https://www.kaggle.com/c/dogs-vs-cats/overview)

El conjunto de datos de Dogs vs Cats fue publicado por Kaggle como parte de una competencia de visión computacional a fines de 2013, cuando las CNNs no eran muy comunes.

Se puede descargar el dataset original en: https://www.kaggle.com/c/dogs-vs-cats/data.

Esta notebook se puede usar con dos conjuntos de datos:

* Usaremos el conjunto de datos original de entrenamiento, dado que contiene las etiquetas de las clases. Este conjunto contiene 25,000 imágenes de perros y gatos (12,500 de cada clase) y tiene un tamaño de 543 MB. Ya se encuentra dividido en *train*, *validation* y *test*. [Download](https://drive.google.com/file/d/1Q3xOfn2Up9uIOLviS66oYH_oFFK-IGpW/view?usp=sharing)

* Usaremos un conjunto reducido de datos, el cual contiene 1000 imágenes de cada clase para entrenamiento, 500 para validación y 500 para prueba. Todos los datos se sacaron del conjunto de entrenamiento original. [Download](https://drive.google.com/file/d/1Ce3u8dwYYriLkz5OpcGn72xIQENIHZX5/view?usp=sharing)

Copiaremos el dataset desde un vínculo de Google Drive

In [ ]:
!pip install -qq gdown

Descargamos el dataset desde Google Drive

In [ ]:
# ----- Versión completa -----
# !gdown --id 1Q3xOfn2Up9uIOLviS66oYH_oFFK-IGpW

# ----- Copia de la versión completa -----
# !gdown 1hchhNQ_3WNncaXVD3kX58EIppcYFt-E2

# ----- Versión reducida -----
!gdown 1Ce3u8dwYYriLkz5OpcGn72xIQENIHZX5

# ----- Copia de la versión reducida -----
# !gdown 1NK9LvrVwsEQM0UHkFHq_GYCF2fjGrwAP

Descomprimimos

In [ ]:
from zipfile import ZipFile

# file_name = '/content/cnn_perros_gatos.zip'
# file_name = '/content/cnn_perros_gatos-copia.zip'
file_name = '/content/cnn_perros_gatos-small.zip'
# file_name = '/content/cnn_perros_gatos-small-copia.zip'

with ZipFile(file_name, 'r') as myzip:
    myzip.extractall()
    print('Listo')

🔵 Exploremos la ruta de archivos del dataset

Veamos el balance de clases

In [ ]:
import os
import matplotlib.pyplot as plt

path = '/content/cnn_perros_gatos/train'

num_train_dogs = len(os.listdir(path + '/dogs'))
num_train_cats = len(os.listdir(path + '/cats'))

print(f'Número de imágenes de entrenamiento de perros: {num_train_dogs}')
print(f'Número de imágenes de entrenamiento de gatos: {num_train_cats}')

ratio = num_train_dogs / (num_train_dogs + num_train_cats)

plt.figure()
plt.suptitle(f'Número de imágenes por clase\nRatio:{round(ratio,3)}')
plt.bar(['Perros', 'Gatos'], [num_train_dogs, num_train_cats])
plt.show()

Exploramos las carpetas de entrenamiento, validación y prueba.

In [ ]:
import os, shutil

print('Para entrenamiento:')

train_dogs = 'cnn_perros_gatos/train/dogs'
print(f'\t{len(os.listdir(train_dogs))} Perros.')
train_cats = 'cnn_perros_gatos/train/cats'
print(f'\t{len(os.listdir(train_cats))} Gatos.')

print('\nPara validación:')
validation_dogs = 'cnn_perros_gatos/validation/dogs'
print(f'\t{len(os.listdir(validation_dogs))} Perros.')
validation_cats = 'cnn_perros_gatos/validation/cats'
print(f'\t{len(os.listdir(validation_cats))} Gatos.')

print('\nPara prueba:')
test_dogs = 'cnn_perros_gatos/test/dogs'
print(f'\t{len(os.listdir(test_dogs))} Perros.')
test_cats = 'cnn_perros_gatos/test/cats'
print(f'\t{len(os.listdir(test_cats))} Gatos.')

Veamos algunas imágenes del dataset, como podemos ver:

* Son archivos jpeg
* Tienen diferentes tamaños

In [ ]:
!pip install -qq ipyplot

In [ ]:
from PIL import Image
import ipyplot
import random, os
import numpy as np

path_1 = '/content/cnn_perros_gatos/train/cats'
path_2 = '/content/cnn_perros_gatos/train/dogs'

filenames_1 = np.random.choice(os.listdir(path_1), 5, replace=False)
filenames_2 = np.random.choice(os.listdir(path_2), 5, replace=False)
# random.sample(os.listdir(path_1), 5)
# filenames_2 = random.sample(os.listdir(path_2), 5)

full_filenames_1 = [os.path.join(path_1, fname) for fname in filenames_1]
full_filenames_2 = [os.path.join(path_2, fname) for fname in filenames_2]

filenames = full_filenames_1 + full_filenames_2
images_list = [Image.open(fname) for fname in filenames]

ipyplot.plot_images(images_list,show_url=False)

Veamos las dimensiones de las imagenes

In [ ]:
import os
import matplotlib.pyplot as plt
from PIL import Image
from seaborn import heatmap
import numpy as np

path = '/content/cnn_perros_gatos/train'

widths = []
heights = []

for folder in os.listdir(path):
    folder_path = os.path.join(path, folder)
    for image in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image)
        img = Image.open(image_path)
        widths.append(img.width)
        heights.append(img.height)

min_width = min(widths)
max_width = max(widths)
min_height = min(heights)
max_height = max(heights)

plt.figure(figsize=(11,5))
plt.subplot(1,2,1)
plt.suptitle('Dimensiones de las imágenes')
plt.hist(widths, bins=20, alpha=0.5, label='Width')
plt.hist(heights, bins=20, alpha=0.5, label='Height')
plt.legend()
plt.subplot(1,2,2)
plt.hist2d(widths, heights, bins=(20, 20), cmap='Blues')
plt.xticks(range(min_width, max_width+1, 100))
plt.yticks(range(min_height, max_height+1, 100))
plt.colorbar()
plt.xlabel('Width')
plt.ylabel('Height')
plt.show()

🔵 ¿Qué retos presentaría este dataset para una MLP como las que hemos definido y usado?

Definimos los directorios de entrenamiento, validación y prueba para usaralos en el resto de la notebook.

In [ ]:
train_dir = 'cnn_perros_gatos/train'
validation_dir = 'cnn_perros_gatos/validation'
test_dir = 'cnn_perros_gatos/test'

Además, probaremos con dos imágenes externas al dataset

In [ ]:
!gdown 1eSWPCWL-mc4ekrjbh5BN25IKlNZfAECF
!gdown 1OEUZgYKM_brFUwjNi1RmMCOLzctdimYK

# Preprocesamiento de datos


Los datos deben formatearse en tensores de punto flotante preprocesados adecuadamente antes de que se introduzcan en la red. En este momento, nuestros datos se encuentran almacenados como archivos JPEG, por lo que los pasos para que puedan ser introducidos en nuestra red son:

* Leer los archivos de imagen.

* Decodificar el contenido JPEG a cuadrículas de píxeles RBG.

* Convertirlos en tensores de punto flotante.

* Volver a escalar los valores de píxeles (entre 0 y 255) al intervalo $[0, 1]$ (las redes neuronales prefieren tratar con valores de entrada pequeños).

Afortunadamente, Keras tiene herramientas para encargarse de estos pasos automáticamente. Keras tiene un módulo con herramientas de ayuda para procesamiento de imágenes, ubicado en **keras.preprocessing.image**. En particular, contiene la clase **ImageDataGenerator**, que permite configurar rápidamente los generadores de Python que pueden convertir automáticamente los archivos de imagen en disco en *batches* de tensores preprocesados.

Definimos una función para obtener los generadores de entranamiento, validación y prueba especificando las rutas de las carpetas y el tamaño de imagen

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import preprocess_input as effnet_preprocess
from tensorflow.keras.applications.resnet import preprocess_input as resnet_preprocess

def get_generators(train_dir,
                  validation_dir,
                  test_dir,
                  img_size,
                  preprocessing_mode='rescaling',
                  model_type=None,
                  augmentation_params=None):
    """
    Crea generadores de datos para entrenamiento, validación y test.

    Args:
        train_dir (str): Directorio de entrenamiento
        validation_dir (str): Directorio de validación
        test_dir (str): Directorio de test
        img_size (tuple): Tamaño de las imágenes (height, width)
        preprocessing_mode (str): Tipo de preprocesamiento ('rescaling', 'model_specific', 'augmenting')
        model_type (str): Tipo de modelo para preprocesamiento específico ('efficientnet', 'resnet', etc.)
        augmentation_params (dict): Parámetros de aumento de datos personalizados

    Returns:
        tuple: (train_generator, validation_generator, test_generator)
    """
    # Configuración base para test y validación (siempre sin aumento de datos)
    test_val_datagen = ImageDataGenerator(rescale=1./255)

    # Configuración para entrenamiento
    if preprocessing_mode == 'rescaling':
        train_datagen = ImageDataGenerator(rescale=1./255)
    elif preprocessing_mode == 'model_specific':
        if model_type == 'efficientnet':
            train_datagen = ImageDataGenerator(preprocessing_function=effnet_preprocess)
            test_val_datagen = ImageDataGenerator(preprocessing_function=effnet_preprocess)
        elif model_type == 'resnet':
            train_datagen = ImageDataGenerator(preprocessing_function=resnet_preprocess)
            test_val_datagen = ImageDataGenerator(preprocessing_function=resnet_preprocess)
    elif preprocessing_mode == 'augmenting':
        if augmentation_params:
            # Usar parámetros personalizados si se proporcionan
            train_datagen = ImageDataGenerator(**augmentation_params)
        else:
            # Configuración por defecto para aumento de datos
            train_datagen = ImageDataGenerator(
                rescale=1./255,
                rotation_range=40,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                horizontal_flip=True,
                fill_mode='nearest'
            )

    # Crear generadores
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=img_size,
        batch_size=20,
        class_mode='binary',
        shuffle=True
    )

    validation_generator = test_val_datagen.flow_from_directory(
        validation_dir,
        target_size=img_size,
        batch_size=20,
        class_mode='binary',
        shuffle=True
    )

    test_generator = test_val_datagen.flow_from_directory(
        test_dir,
        target_size=img_size,
        batch_size=20,
        class_mode='binary',
        shuffle=False
    )

    return train_generator, validation_generator, test_generator

In [ ]:
img_size = (16,16)

train_generator, validation_generator, test_generator = get_generators(train_dir,
                                                                       validation_dir,
                                                                       test_dir,
                                                                       img_size,
                                                                       preprocessing_mode='rescaling',
                                                                       model_type=None,
                                                                       augmentation_params=None
                                                                       )

In [ ]:
import numpy as np

train_labels = train_generator.classes

np.unique(train_labels,return_counts=True)

In [ ]:
print(train_generator.class_indices)
print(validation_generator.class_indices)
print(test_generator.class_indices)


* Vamos a revisar la salida de uno de estos generadores: produce batches de imágenes de 150 x 150 RGB (con la forma (20, 150, 150, 3)) y etiquetas binarias (con la forma (20,)). 20 es el número de muestras en cada batch (el tamaño del batch).
* Como el generador genera estos batches de forma indefinida (i.e. recorre sin fin las imágenes presentes en la carpeta que se le indicó), se necesita romper el loop de iteración en algún punto. A continuación podemos ver cómo es cada corrida (batch/step) que proporciona el generador.

In [ ]:
x_train, y_train = next(train_generator)
print("Labels in batch:", y_train)
print("Shape:",x_train.shape)
print("Number of class 0:", sum(y_train==0))
print("Number of class 1:", sum(y_train==1))

x_test, y_test = next(test_generator)
print("Labels in batch:", y_test)
print("Shape:",x_test.shape)
print("Number of class 0:", sum(y_test==0))
print("Number of class 1:", sum(y_test==1))

x_val, y_val = next(validation_generator)
print("Labels in batch:", y_val)
print("Shape:",x_val.shape)
print("Number of class 0:", sum(y_val==0))
print("Number of class 1:", sum(y_val==1))

* Vamos a proceder a entrenar nuestro modelo con los datos usando el generador. Debido a que los datos se generan infinitamente, el generador necesita saber cuántas muestras extraer antes de declarar una época finalizada. Esta es la función del argumento **steps_per_epoch**

* En este caso, **steps_per_epoch** corresponde al número de batches que requiere el generador para leer el conjunto de datos completo. Sólo después de haber solicitado este número de batches, el proceso de ajuste de nuestro modelo pasará a la siguiente época. **steps_per_epoch** corresponde a el número de pasos de descenso del gradiente. En nuestro caso, cada batch tiene un tamaño de 20 muestras, por lo que tomará 100 pasos (batches) hasta que cubramos las 2,000 muestras de nuestra base de datos.

* Como siempre, uno puede pasar un argumento llamado **validation_data**. Es importante destacar que este argumento puede ser un generador de datos en sí mismo, pero también podría ser una tupla de arreglos Numpy. Si se pasa un generador como **validation_data**, entonces se espera que este generador produzca batches de datos de validación sin fin, y por lo tanto también se debe especificar el argumento **validation_steps**, que le dice al proceso cuántos batches debe extraer del generador de validación para su evaluación.

Definamos funciones para graficar las curvas de entrenamiento y mostrar el rendimiento en el conjunto de prueba

In [ ]:
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from seaborn import heatmap

def plot_training_curves(history):
    plt.figure(figsize=(11,5))
    plt.subplot(1,2,1)
    plt.title("Validation and Training Loss",fontsize=14)
    plt.plot(history.history['loss'], label='train')
    plt.plot(history.history['val_loss'], label='validation')
    plt.legend()
    plt.subplot(1,2,2)
    plt.title("Validation and Training Accuracy",fontsize=14)
    plt.plot(history.history['accuracy'], label='train')
    plt.plot(history.history['val_accuracy'], label='validation')
    plt.legend()
    plt.show()

def evaluate(model,X,y,classes_names):
    if len(classes_names) > 2:
        y_pred_proba = model.predict(X)
        y_pred = np.argmax(y_pred_proba,axis=1)
    elif len(classes_names) == 2:
        y_pred_proba = model.predict(X)
        y_pred = np.where(y_pred_proba > 0.5, 1, 0).reshape(-1)
    print(f"Accuracy: {accuracy_score(y,y_pred)}")
    print(f"F1 Score: {f1_score(y,y_pred,average='macro')}")
    cm = confusion_matrix(y_pred=y_pred,y_true=y)
    plt.figure()
    heatmap(cm,
            fmt='g',
            annot=True,
            xticklabels=classes_names,
            yticklabels=classes_names,
            cmap='Blues'
            )
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

# Modelo 0: MLP

In [ ]:
train_generator, validation_generator, test_generator = get_generators(train_dir,
                                                                       validation_dir,
                                                                       test_dir,
                                                                       img_size=(16,16),
                                                                       preprocessing_mode='rescaling',
                                                                       model_type=None,
                                                                       augmentation_params=None
                                                                       )

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Input, Dropout
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

input_shape = (16,16,3)

model = Sequential([
    Input(shape=input_shape),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.summary()

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss',
                               patience=5,
                               restore_best_weights=True
                               )

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=4,
                              min_lr=1e-5)

In [ ]:
train_steps = train_generator.samples // train_generator.batch_size
val_steps = validation_generator.samples // validation_generator.batch_size

print(f"Número de pasos de entrenamiento: {train_steps}")
print(f"Número de pasos de validación: {val_steps}")

history = model.fit(
      train_generator,
      steps_per_epoch=train_steps,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=val_steps,
      callbacks=[early_stopping, reduce_lr]
      )

In [ ]:
plot_training_curves(history)

In [ ]:
test_generator.reset()
classes_names = list(train_generator.class_indices.keys())
evaluate(model,test_generator,test_generator.classes,classes_names)

In [ ]:
model.evaluate(test_generator)

# Modelo 1

Definamos ahora un modelo con arquitectura CNN. Usaremos las capas [`Conv2D`](https://keras.io/api/layers/convolution_layers/convolution2d/) para las operaciones de convolución y [`MaxPooling2D`](https://keras.io/api/layers/pooling_layers/max_pooling2d/) para el pooling.

Pero antes, obtengamos los generadores de imágenes con un tamaño mayor

In [ ]:
img_size = (150,150)

train_generator, validation_generator, test_generator = get_generators(train_dir,
                                                                       validation_dir,
                                                                       test_dir,
                                                                       img_size, # Aumentamos el tamaño de imágenes
                                                                       preprocessing_mode='rescaling', # Sólo re-escalamos
                                                                       model_type=None,
                                                                       augmentation_params=None # Sin aumento de datos
                                                                       )

<p align="center">
  <img src="https://drive.google.com/uc?id=1bzFBdsAq40yN95k2pA5X2OGsXf-40v0t" width="600" />
</p>


**Importante**: Observa la elección de los hiperparámetros `padding="same"` y `strides=1`. Esta elección asegura que las salidas de cada capa convolucional tenga las mismas dimensiones que las entradas.

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Input
from keras.models import Sequential

model = Sequential([
    Input(shape=(150, 150, 3)),
    Conv2D(32, 3, activation='relu',
                           input_shape=(150, 150, 3)),
    MaxPooling2D(),
    Conv2D(64, 3, activation='relu'),
    MaxPooling2D(),
    Conv2D(128, 3, activation='relu'),
    MaxPooling2D(),
    Conv2D(128, 3, activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [ ]:
model.summary()

* NOTA que comenzamos con imágenes de tamaño 150 x 150 (una elección de tamaño arbitraria) y terminamos con mapas de características de tamaño 7 x 7 justo antes de la capa de *flatten*.
* En realidad las imágenes de entrada tienen tamaños diversos (desconocidos), pero afortunadamente Keras nos puede ayudar a pre-procesarlas.

Compilamos el modelo

In [ ]:
from keras.optimizers import Adam

opt = Adam(learning_rate=1e-4)

model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])

## Entrenamiento del modelo

Tarda alrededor de 3 minutos



In [ ]:
history = model.fit(
      train_generator,
      steps_per_epoch=100,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=50)

## Rendimiento del modelo

Guardamos el modelo

In [ ]:
model.save('cnn_perros_gatos_model_1.keras')

Grafiquemos las curvas de aprendizaje

In [ ]:
plot_training_curves(history)

**Overfitting**... ¡Tenemos muy pocos ejemplos!

In [ ]:
model.evaluate(test_generator)

Evaluemos el desempeño del modelo a detalle

In [ ]:
test_generator.reset()
classes_names = list(train_generator.class_indices.keys())
evaluate(model,test_generator,
         test_generator.classes,
         classes_names)

# Modelo 2: Aumento de datos

## Aumento de Datos

* El efecto de sobreajuste ocurre cuando se tienen muy pocas muestras de las que aprender, lo que nos impide entrenar un modelo capaz de generalizar a nuevos datos. Si tuviesemos datos infinitos, nuestro modelo estaría expuesto a todos los aspectos posibles de la distribución de datos en cuestión y nunca se sobreajustaría nuestro modelo.

* El aumento de datos adopta el enfoque de generar más datos de entrenamiento a partir de muestras de entrenamiento existentes, al "aumentar" las muestras a través de una serie de transformaciones aleatorias que producen imágenes de apariencia creíble. El objetivo es que durante el tiempo de entrenamiento, nuestro modelo nunca vea exactamente la misma imagen dos veces. Esto ayuda a que el modelo se exponga a más aspectos de los datos y generalice mejor.

* En Keras, esto se puede hacer configurando una serie de transformaciones aleatorias que se realizarán en las imágenes leídas por nuestra instancia de ImageDataGenerator.

* Vamos a comenzar por aumentar una imagen.



In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

Las opciones anteriores son solo algunas de las opciones disponibles.

* **rotation_range** es un valor en grados (0-180), un rango dentro del cual girar las imágenes de forma aleatoria.

* **width_shift** y **height_shift** son rangos expresados como una fracción del ancho o altura total de la imagen, dentro de los cuales se pueden trasladar vertical u horizontalmente de forma aleatoria a las imágenes.

* **shear_range** aplica aleatoriamente transformaciones de corte.

* **zoom_range** aplica acercamientos aleatorios dentro de las imágenes.

* **horizontal_flip** Voltea de forma aleatoria la mitad en las imágenes horizontalmente.

* **fill_mode** es la estrategia utilizada para rellenar píxeles creados, que pueden aparecer después de una rotación o un cambio de ancho / altura.

Generamos 25 imágenes modificadas a partir de una misma imagen

In [ ]:
from keras import utils
import random

train_cats_dir = '/content/cnn_perros_gatos/train/cats' # El directorio donde están las imágenes de gatos de entrenamiento
fnames = [os.path.join(train_cats_dir, fname) for fname in os.listdir(train_cats_dir)]
img_path = random.sample(fnames, 1)[0] # Escogemos una imagen al azar para aplicar el "aumentado de datos"
img = utils.load_img(img_path, target_size=(150, 150)) # Leemos la imagen y la redimensionamos.
x = utils.img_to_array(img) # Leemos la imagen y la redimensionamos.
x = x.reshape((1,) + x.shape) # Redimensionamos el arreglo a (1, 150, 150, 3)

'''
El comando .flow () genera batches de imágenes transformadas aleatoriamente
Con el "for" de abajo estaremos en un loop indefinidamente,
Necesitamos 'romper' el loop en algún momento
'''

fig, axs = plt.subplots(5,5,figsize=(10,10))
k = 0
for batch in datagen.flow(x, batch_size=1):
    i = k//5
    j = k%5
    axs[i,j].imshow(utils.array_to_img(batch[0]))
    axs[i,j].axis('off')
    k += 1
    if k == 25:
        break
plt.tight_layout()
plt.show()

* Si entrenamos una nueva red neuronal utilizando esta configuración de aumento de datos, nuestra red nunca verá dos veces la misma entrada, pues a cada nueva imagen se le aplica transformaciones aleatorias dentro de ciertos rangos.

* Sin embargo, las entradas que ves están aún muy interrelacionadas, ya que provienen de un pequeño número de imágenes originales: **no podemos producir nueva información, sólo podemos mezclar la información existente**.

* Dado que esto podría no ser suficiente para librarnos del sobreajuste. Para mitigarlo aún más, también agregaremos una capa de Dropout a nuestro modelo, justo antes de la etapa del clasificador densamente conectado (fully-connected).

Definimos los nuevos generadores de imágenes

In [ ]:
train_generator, validation_generator, test_generator = get_generators(train_dir,
                                                                       validation_dir,
                                                                       test_dir,
                                                                       img_size=(150,150),
                                                                       preprocessing_mode='augmenting',
                                                                       model_type=None,
                                                                       augmentation_params=None)

Definimos la misma red CNN, con dropout.

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Input
from keras.models import Sequential

model = Sequential([
    Input(shape=(150, 150, 3)),
    Conv2D(32, 3, activation='relu', name='Convolution1'),
    MaxPooling2D(name='MaxPooling1'),
    Conv2D(64, 3, activation='relu',name='Convolution2'),
    MaxPooling2D(name='MaxPooling2'),
    Conv2D(128, 3, activation='relu',name='Convolution3'),
    MaxPooling2D(name='MaxPooling3'),
    Conv2D(128, 3, activation='relu',name='Convolution4'),
    MaxPooling2D(name='MaxPooling4'),
    Flatten(name='Flatten'),
    Dropout(0.1,name='DropOut'), # 0.5
    Dense(512, activation='relu',name='Densa'),
    Dense(1, activation='sigmoid',name='Salida')
])

model.summary()

In [ ]:
from keras.optimizers import RMSprop

opt = RMSprop(learning_rate=1e-4)

In [ ]:
model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])

## Entrenamiento

Entrenemos el modelo.

**Observación**: Para mejores resultados entrenar durante 100 épocas (tarda alrededor de 30 minutos). Por cuestiones de tiempo, entrenamos con 30 épocas (tarda alrededor de 8 minutos). Con ambos obtendremos un poco más de 80% de accuracy.

In [ ]:
history = model.fit(
                train_generator,
                steps_per_epoch=100,
                epochs=30,
                validation_data=validation_generator,
                validation_steps=50,
                verbose=1)

Podríamos guardar el modelo

In [ ]:
model.save('cnn_perros_gatos_model2_30_epochs.keras')

Veamos las curvas de entrenamiento

In [ ]:
plot_training_curves(history)

Evaluemos el desempeño

In [ ]:
model.evaluate(test_generator)

In [ ]:
test_generator.reset()
class_names = list(train_generator.class_indices.keys())
evaluate(model,test_generator,
         test_generator.classes,
         classes_names)

## ¿Qué pasa si entrenamos con más épocas y todo el dataset?

A continuación se muestras las gráficas de entrenamiento y la matriz de confusión con un modelo más grande, entrenado durante cerca de 50 épocas, usando todo el conjunto de entrenamiento completo.

El accuracy en el conjunto de prueba fue de 87%.

Podemos descargar este modelo ya entrenado de Google Drive




<p align="center">
  <img src="https://drive.google.com/uc?id=1l4zeHmnDvsxhrHlMH0xgzT9BaYfa2oMs" width="600" />
</p>


<p align="center">
  <img src="https://drive.google.com/uc?id=1eNMacyz1KA0WAwJW3fn1ktfm5_MrpJem" width="600" />
</p>

In [ ]:
!gdown 13L5oCFXIgw22FR8irsuwNHvnzekA7bGd

In [ ]:
from keras.models import load_model

model = load_model('/content/cnn_perros_gatos_improved.keras')
model.summary()

In [ ]:
train_dir = 'cnn_perros_gatos/train'
validation_dir = 'cnn_perros_gatos/validation'
test_dir = 'cnn_perros_gatos/test'

train_generator, validation_generator, test_generator = get_generators(train_dir,
                                                                       validation_dir,
                                                                       test_dir,
                                                                       img_size=(150,150),
                                                                       preprocessing_mode='rescaling',
                                                                       model_type=None,
                                                                       augmentation_params=None)

In [ ]:
test_generator.reset()
classes_names = list(train_generator.class_indices.keys())
evaluate(model,test_generator,
         test_generator.classes,
         classes_names)

In [ ]:
model.evaluate(test_generator)

🔵 ¿Qué imágenes son las que está confundiendo el modelo?

In [ ]:
test_generator.reset()
y_pred_proba = model.predict(test_generator)
y_pred = np.where(y_pred_proba > 0.5, 1, 0).reshape(-1)

false_positives_idxs = np.where((y_pred == 1) & (test_generator.classes == 0))[0]
false_negatives_idxs = np.where((y_pred == 0) & (test_generator.classes == 1))[0]

print(f"False positives: {false_positives_idxs.shape[0]}")
print(f"False negatives: {false_negatives_idxs.shape[0]}")

In [ ]:
import imageio
import matplotlib.pyplot as plt

# Diccionario con el nombre de cada clase
test_generator.class_indices
idxs_to_classes = {v:k for k,v in test_generator.class_indices.items()}

# Escogemos un ejemplo de falso positivo y uno de falso negativo
fp_idx = np.random.choice(false_positives_idxs,size=1)
fn_idx = np.random.choice(false_negatives_idxs,size=1)

# Obtenemos los nombres de archivo de esos índices
test_generator.reset()
fp_filename = test_generator.filenames[fp_idx[0]]
fn_filename = test_generator.filenames[fn_idx[0]]

# Leemos las imágenes
fp_img = imageio.v2.imread(os.path.join(test_dir,fp_filename))
fn_img = imageio.v2.imread(os.path.join(test_dir,fn_filename))

# Mostramos las imágenes
fig, axs = plt.subplots(1,2,figsize=(10,5))
axs[0].imshow(fp_img)
axs[0].set_title(f'Prediction: {idxs_to_classes[y_pred[fp_idx][0]]}')
axs[0].axis('off')
axs[1].imshow(fn_img)
axs[1].set_title(f'Prediction: {idxs_to_classes[y_pred[fn_idx][0]]}')
axs[1].axis('off')
plt.show()

In [ ]:
from imageio.v2 import imread
import matplotlib.pyplot as plt
import os

dog_path = 'dc_dog.jpg'
cat_path = 'dc_cat.jpg'

dog_img = imread(dog_path)
cat_img = imread(cat_path)

plt.figure()
plt.subplot(1,2,1)
plt.imshow(dog_img)
plt.axis('off')
plt.subplot(1,2,2)
plt.imshow(cat_img)
plt.axis('off')
plt.show()

In [ ]:
from tensorflow.keras.preprocessing import image

# Función para cargar y preprocesar una imagen
def load_and_preprocess_image(img_path, target_size=(150, 150)):
    # Cargar imagen y redimensionar a 150x150
    img = image.load_img(img_path, target_size=target_size)
    # Convertir a array numpy
    img_array = image.img_to_array(img)
    # Normalizar valores de píxeles al rango [0,1]
    img_array = img_array / 255.0
    return img_array

In [ ]:
dog_img = load_and_preprocess_image(dog_path)
cat_img = load_and_preprocess_image(cat_path)

# Crear el tensor combinando ambas imágenes
images_tensor = np.stack([dog_img, cat_img], axis=0)

print(images_tensor.shape)

preds = model.predict(images_tensor)
print(f"Salida de la red:\t{preds.reshape(-1,)}\n")

predictions = np.where(preds > 0.5, 1, 0).reshape(-1)

idxs_to_classes = {v:k for k,v in test_generator.class_indices.items()}

plt.figure()
plt.subplot(1,2,1)
plt.imshow(dog_img)
plt.axis('off')
plt.title(f'Prediction: {idxs_to_classes[predictions[0]]}')
plt.subplot(1,2,2)
plt.imshow(cat_img)
plt.axis('off')
plt.title(f'Prediction: {idxs_to_classes[predictions[1]]}')
plt.show()


# ⚡ Usando la red para obtener features de las imágenes: Embeddings

En esta parte de la notebook ilustraremos cómo la parte convolucional de las redes CNN se puede ver como un método de extracción de features. Es decir, podemos ver al bloque convolucional como un método que convierte cada imagen en un vector, de una *buena* manera.

Para esto, usaremos el modelo CNN pre-entrenado con el dataset complejo.

In [ ]:
!gdown 13L5oCFXIgw22FR8irsuwNHvnzekA7bGd

In [ ]:
from keras.models import load_model

model = load_model('/content/cnn_perros_gatos_improved.keras')
model.summary()

Podemos acceder a las distintas capas

In [ ]:
len(model.layers)

Ahora, definimos un modelo de Keras que será el mismo modelo pre-entrenado pero sin la capa de salida.

Para esto, usamos la clase `Model` de Keras.

<p align="center">
  <img src="https://drive.google.com/uc?id=1lbLQ7D1_QElq_iTscpQ_rjXPWJ2uPke3" width="600" />
</p>

In [ ]:
from keras.models import Model

first_layer = model.layers[0] # Capa de entrada del modelo original

features_layer = model.layers[20] # Si quieremos la salida justo antes de la capa de salida
# features_layer = model.layers[15]  # Si queremos la salida de la parte convolucional

# Creamos el modelo especificando la(s) entrada(s) y salida(s)
features_model = Model(inputs=first_layer.input, outputs=features_layer.output)

Definimos los generadores

In [ ]:
train_generator, validation_generator, test_generator = get_generators(train_dir,
                                                                       validation_dir,
                                                                       test_dir,
                                                                       img_size=(150,150),
                                                                       preprocessing_mode='rescaling',
                                                                       model_type=None,
                                                                       augmentation_params=None
                                                                       )

Como necesitamos obtener predicciones sobre el conjunto de prueba, definamos la siguiente función:

In [ ]:
import numpy as np
from tqdm import tqdm  # Opcional: para barra de progreso

def get_embeddings_with_shuffled_generator(model, generator):
    """
    Obtiene embeddings y etiquetas REALES de un generador (incluso con shuffle=True).

    Args:
        model: Modelo de Keras que devuelve embeddings (sin capa softmax).
        generator: Generador de imágenes (DirectoryIterator).

    Returns:
        tuple: (y_true, embeddings) - etiquetas y embeddings alineados.
    """
    generator.reset()  # Reinicia el generador
    y_true = []
    embeddings = []

    # Iterar sobre TODOS los batches del generador
    for _ in tqdm(range(len(generator))):
        x_batch, y_batch = next(generator)
        y_true.extend(y_batch)
        batch_embeddings = model.predict(x_batch, verbose=0)
        embeddings.extend(batch_embeddings)

    # Convertir a numpy array
    y_true = np.array(y_true)
    embeddings = np.array(embeddings)

    return y_true, embeddings

Obtenemos las features para el conjunto de entrenamiento y prueba, pasandolas por este nuevo modelo `features_model`

In [ ]:
y_train, train_features = get_embeddings_with_shuffled_generator(features_model, train_generator)

In [ ]:
test_generator.reset()
y_test = test_generator.classes

test_generator.reset()
test_features = features_model.predict(test_generator)
print(test_features.shape)

A partir de este punto, ya podemos usar estas features como features para cualquier método de Machine Learning (clásico o profundo).

In [ ]:
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score


clfs = [SVC(),
        DecisionTreeClassifier(max_depth=10),
        RandomForestClassifier(n_estimators=50,max_depth=10),
        KNeighborsClassifier()]
names = [x.__class__.__name__ for x in clfs]

for clf, name in zip(clfs, names):
    clf = Pipeline([('scaler', StandardScaler()), (name, clf)])
    clf.fit(train_features, train_generator.classes)
    y_pred_train = clf.predict(train_features)
    y_pred_test = clf.predict(test_features)
    print(f'{name} - Train Accuracy: {accuracy_score(y_true=y_train, y_pred=y_pred_train)}')
    print(f'{name} - Test Accuracy: {accuracy_score(y_true=y_test, y_pred=y_pred_test)}')


🔵 Tenemos mejores métricas que con la red MLP. Esto nos dice que la parte convolucional de la red es un buen método para extraer features.

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

pca = PCA(n_components=2)
train_pca = pca.fit_transform(train_features)

plt.figure()
plt.scatter(train_pca[:,0], train_pca[:,1], c=train_generator.classes)
plt.legend()
plt.show()

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

tsne = TSNE(n_components=2, random_state=0)
train_tsne = tsne.fit_transform(train_features)

plt.figure()
plt.scatter(train_tsne[:,0], train_tsne[:,1], c=train_generator.classes)
plt.show()

## Usando un modelo pre-entrado especializado

**EfficientNetB0** es una red neuronal convolucional pre-entrenada en el conjunto de datos ImageNet, que puede clasificar imágenes en 1000 categorías de objetos. Es una de las variantes de la familia EfficientNet, conocida por su eficiencia en términos de parámetros y cálculos, logrando un buen equilibrio entre precisión y tamaño.


<p align="center">
  <img src="https://drive.google.com/uc?id=1y5dsxf3gUaaVaodEcwkuovw-zgrIDuXI" width="600" />
</p>

[Documentación](https://keras.io/api/applications/efficientnet/)

In [ ]:
train_dir = 'cnn_perros_gatos/train'
validation_dir = 'cnn_perros_gatos/validation'
test_dir = 'cnn_perros_gatos/test'

generators = get_generators(train_dir, validation_dir, test_dir,
                            img_size=(224,224),
                            preprocessing_mode='model_specific',
                            model_type='efficientnet',
                            augmentation_params=None
                            )

train_generator_en, validation_generator_en, test_generator_en = generators

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing import image
import numpy as np

# Cargar el modelo preentrenado (sin la capa superior 'softmax')
model_en = EfficientNetB0(weights='imagenet', include_top=False, pooling='avg')

y_train, train_features_EN = get_embeddings_with_shuffled_generator(model_en, train_generator_en)
print("Train embedding shape:", train_features_EN.shape)

test_generator_en.reset()
y_test = test_generator_en.classes
test_features_EN = model_en.predict(test_generator_en)
print("Test embedding shape:", test_features_EN.shape)

In [ ]:
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA


clfs = [SVC(),
        DecisionTreeClassifier(max_depth=5),
        RandomForestClassifier(n_estimators=30,max_depth=5),
        KNeighborsClassifier()]
names = [x.__class__.__name__ for x in clfs]

for clf, name in zip(clfs, names):
    clf = Pipeline([('scaler', StandardScaler()),
                    ('pca', PCA(n_components=100)),
                    (name, clf)])
    clf.fit(train_features_EN, y_train)
    y_pred_train = clf.predict(train_features_EN)
    y_pred_test = clf.predict(test_features_EN)
    print(f'{name} - Train Accuracy: {accuracy_score(y_true=y_train, y_pred=y_pred_train)}')
    print(f'{name} - Test Accuracy: {accuracy_score(y_true=y_test, y_pred=y_pred_test)}')

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

pca = PCA(n_components=2)
train_pca = pca.fit_transform(train_features_EN)
test_pca = pca.transform(test_features_EN)

plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.scatter(train_pca[:,0], train_pca[:,1], c=y_train)
plt.subplot(1,2,2)
plt.scatter(test_pca[:,0], test_pca[:,1], c=y_test)
plt.show()

In [ ]:
model_ml = Pipeline([('scaler', StandardScaler()),
                    ('pca', PCA(n_components=100)),
                    ('clf', SVC())])

model_ml.fit(train_features_EN, y_train)

evaluate(model_ml, test_features_EN, y_test, ['cat','dog'])

Veamos algunas de las clasificaciones incorrectas

In [ ]:
from imageio.v2 import imread
import matplotlib.pyplot as plt
import os

y_test_pred = model_ml.predict(test_features_EN)

mistakes_idxs = np.where(y_test_pred != y_test)[0]

idxs = np.random.choice(mistakes_idxs,size=3)

fig, axs = plt.subplots(1,3,figsize=(15,5))
for i,idx in enumerate(idxs):
    img = imread(os.path.join(test_dir,test_generator_en.filenames[idx]))
    axs[i].imshow(img)
    axs[i].set_title(f'Prediction: {int(y_test_pred[idx])}')
    axs[i].axis('off')
plt.show()

# 🔽 Visualizando los mapas de características de una red neuronal convolucional

Algunos señalan que los modelos de aprendizaje profundo funcionan como "cajas negras", pues aprenden representaciones que son difíciles de extraer y presentar de una forma legible para el ser humano.

Si bien esto es parcialmente cierto para algunos tipos de modelos de aprendizaje profundo, definitivamente no lo es para las redes convolucionales (*CNN*). Las representaciones aprendidas por las redes convolucionales son altamente susceptibles de visualización, en gran parte porque son representaciones de conceptos visuales. Desde 2013, se ha desarrollado una amplia gama de técnicas para visualizar e interpretar estas representaciones. No exploraremos todas ellas, pero mencionaremos tres de las más accesibles y útiles:



*   **Visualización de las salidas intermedias de una *CNN*  ("activaciones intermedias")**. Este método es útil para entender cómo las capas sucesivas de una red convolucional transforman su entrada y para obtener una noción de la función de los filtros individuales en una red convolucional .

*   **Visualización de los  filtros en una CNN**. Este método es útil para entender con precisión a qué patrón o concepto visual es receptivo cada filtro en una red convolucional.

*   **Visualización de los mapas de calor de activación por clase en una imagen**. Este método es útil para entender qué parte de una imagen se identificó como perteneciente a una clase determinada y, por lo tanto, permite localizar objetos en imágenes.

En este ejercicio, abordaremos únicamente el primer método, la visualización de las activaciones intermedias o mapas de características. Para ello, usaremos la CNN que entrenamos anteriormente.



Seleccionaremos una imagen de entrada, puede ser cualquier imagen del **conjunto de test**. Por ser del conjunto de test, no forma parte de las imágenes sobre las que se entrenó la red.



In [ ]:
from keras import utils
import numpy as np

# ----- Para el conjunto de datos completo ----
# img_path = 'cnn_perros_gatos/test/cats/cat.147.jpg'   # Una imágen de un gato
# img_path = 'cnn_perros_gatos/test/dogs/dog.1517.jpg'  # Una imágen de un perro

# ----- Para el conjunto de datos reducido ----
img_path = '/content/cnn_perros_gatos/test/cats/cat.10128.jpg'
# img_path = '/content/cnn_perros_gatos/test/dogs/dog.10086.jpg'

# ----- Preprocesamos la imagen en un tensor 4D

img = utils.load_img(img_path, target_size=(150, 150))
img_tensor = utils.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0)

# ----- Debemos recordar que el modelo fue entrenado con imagenes de entrada preprocesadas de la siguiente manera:
img_tensor /= 255.

# Debemos ver que su forma es de (1, 150, 150, 3)
print(img_tensor.shape)

Mostramos la imágen

In [ ]:
import matplotlib.pyplot as plt

# # Para usar alguna de las imagenes reales
# img_tensor = dog_img.reshape((1,150,150,3))
# img_tensor = cat_img.reshape((1,150,150,3))

plt.imshow(img_tensor[0])
plt.axis('Off')
plt.show()

* Para extraer los mapas de características que queremos visualizar, crearemos un modelo de Keras que toma lotes ó *batches* de imágenes como entrada y genera las activaciones de todas las capas de convolución y *pooling*.
* Para ello, utilizaremos la clase de Keras **Model**, que ya vimos anteriormente. Un **model** se instancia mediante dos argumentos: un tensor de entrada (o lista de tensores de entrada) y un tensor de salida (o lista de tensores de salida). La clase resultante es un modelo de Keras, igual que los modelos secuenciales (Sequential models) que ya estudiamos, que mapea las entradas especificadas a las salidas especificadas. Lo que distingue a la clase **Model** es que permite modelos con múltiples salidas, a diferencia de **Sequential**.

In [ ]:
!gdown 13L5oCFXIgw22FR8irsuwNHvnzekA7bGd

In [ ]:
from keras.models import load_model

model = load_model('/content/cnn_perros_gatos_improved.keras')
model.summary()

Veamos la forma de la salida de la primer capa

In [ ]:
model.layers[0].output

Veamos las formas y nombres de las capas del modelo. Además, identifiquemos los índices de las capas convolucionales

In [ ]:
conv_layers_idxs = []
conv_layers_names = []

for idx, layer in enumerate(model.layers):
    print(f"{idx}\t{layer.name}")
    if layer.name.startswith('conv'):
        conv_layers_idxs.append(idx)
        conv_layers_names.append(layer.name)

print(conv_layers_idxs)

Creemos un modelo tipo `Model` con tantas salidas como capas convolucionales tenemos.

In [ ]:
from keras.models import Model

# Extraemos las salidas de las 8 capas superiores:
layer_outputs = [model.layers[j].output for j in conv_layers_idxs]

first_layer = model.layers[0]

# Creamos un modelo que devolverá estas salidas, dada la entrada al modelo:
activation_model = Model(inputs=first_layer.input, outputs=layer_outputs)

* Cuando se introduce una imagen como entrada a la red, este modelo devuelve los valores de las activaciones de las capas del modelo original. Hasta antes de esta sección del ejercicio, el modelo que se presentó sólo tenía exactamente una entrada y una salida. Ahora estamos introduciendo el concepto de un modelo con múltiples salidas.

* En el caso general, un modelo podría tener cualquier número de entradas y salidas. Este último modelo tiene una entrada y varias salidas, una salida por capa de convolución. Aunque, cada capa de convolución tiene varias dimensiones.

In [ ]:
# Esto devolverá una lista de arreglos de Numpy: Un arreglo por capa de activación
activations = activation_model.predict(img_tensor)

In [ ]:
print(f"Número de salidas del modelo: {len(activations)}")
for j,activation in enumerate(activations):
    print(f"Forma de la activación {j}: {activation.shape}")

Comparar con el modelo original

In [ ]:
model.summary()

Es un mapa de características con una dimensión de 148 x 148 con 32 canales o profundidad.

Vamos a visualizar uno de estos canales de ese mapa de características.

In [ ]:
import matplotlib.pyplot as plt

first_layer_activation = activations[0]

_, w, h, ch = first_layer_activation.shape
print(f"La salida son {ch} imágenes de {w}x{h}")
print(f"Shape: {first_layer_activation.shape}")

num_canal = 18

plt.matshow(first_layer_activation[0, :, :, num_canal], cmap='plasma')
plt.axis('Off')
plt.suptitle(f"Canal {num_canal}")
plt.show()

Finalmente, vamos a desplegar un gráfico completo de todas las activaciones en la red. En otras palabras, vamos a extraer y mostrar cada canal presente en cada uno de los mapas de características. Apilaremos los resultados secuencialmente, con los canales colocados uno junto al otro.



Primero, guardamos los nombres de las capas

In [ ]:
# Recordemos los nombres de las capas convolucionales del modelo
print(conv_layers_names)

# Especificamos cuántas imagenes por cada renglón
images_per_row = 16

In [ ]:
for k,(layer_name, layer_activation) in enumerate(zip(conv_layers_names, activations)):
    # Este es el número de canales presentes en un mapa de características
    n_features = layer_activation.shape[-1]

    # El mapa de características tiene la forma: (1, size, size, n_features)
    size = layer_activation.shape[1]

    # Vamos a colocar los canales de activación en esta matriz
    n_cols = n_features // images_per_row
    display_grid = np.zeros((size * n_cols, images_per_row * size)) # Aquí vamos a poner toda la imagen de salida de la capa

    # Colocaremos cada mapa en esta gran malla horizontal
    for col in range(n_cols):
        for row in range(images_per_row):
            channel_image = layer_activation[0, :, :, col * images_per_row + row]

            '''
            Este proceso toma la salida de la capa convolucional
            (que pueden tener cualquier rango de valores) y las
            transforma en una imagen visualizable con buen contraste
            y en el formato estándar de píxeles.
            '''
            channel_image -= channel_image.mean() # Centrado en cero
            channel_image /= channel_image.std() # Normalización por desviación estándar
            channel_image *= 64 # Escalado de contraste
            channel_image += 128 # Desplazamiento a rango positivo
            channel_image = np.clip(channel_image, 0, 255).astype('uint8') # Limitación al rango válido
            display_grid[col * size : (col + 1) * size,
                         row * size : (row + 1) * size] = channel_image # Llenamos la parte correspondiente de la imagen

    # Mostramos los mapas en la malla
    scale = 1. / size
    plt.figure(figsize=(scale * display_grid.shape[1],
                        scale * display_grid.shape[0]))
    plt.title(layer_name)
    plt.grid(False)
    plt.axis('Off')
    plt.imshow(display_grid, aspect='auto', cmap='viridis')
    plt.savefig(f"mascara-{k+1}.png")

plt.show()

# Observaciones

Del gráfico de mapas de características podemos notar lo siguiente:

* La primera capa de la red actúa como una colección de varios detectores de borde. En esa etapa, las activaciones aún retienen casi toda la información presente en la imagen inicial.

* A medida que avanzamos en profundidad, las activaciones se vuelven cada vez más abstractas y menos interpretables visualmente. Se comienzan a codificar conceptos de nivel superior como "oreja de gato" u "ojo de gato". Las representaciones superiores llevan cada vez menos información sobre el contenido visual de la imagen, y cada vez más información relacionada con la clase de la imagen.

* La escasez de activaciones aumenta con la profundidad de la red: en la primera capa, todos los filtros se activan mediante la imagen de entrada, pero en las siguientes capas, más y más canales de activación están en blanco. Esto significa que el patrón codificado por el filtro no se encuentra en la imagen de entrada.

# Comentarios Finales

Acabamos de evidenciar un hecho muy importante de las representaciones aprendidas por las redes neuronales profundas: las características extraídas por una capa se vuelven cada vez más abstractas con la profundidad de la red.

Las activaciones de las capas superiores contienen cada vez menos información sobre la entrada específica que se está viendo y más información sobre el objetivo (en el caso de este ejemplo, la clase de la imagen: gato o perro). Una red neuronal profunda actúa efectivamente como un *pipeline* (tubería) que destila la información, con datos en crudo que entran (en nuestro caso, imágenes RBG) y se transforman repetidamente de tal forma que la información irrelevante es filtrada (por ejemplo, la apariencia visual específica de la imagen) mientras que la información útil es magnificada y refinada (por ejemplo, la clase de la imagen).

Esto es análogo a la forma en que los humanos y los animales perciben el mundo: después de observar una escena durante unos segundos, un humano puede recordar qué objetos abstractos estaban presentes en él (por ejemplo, una bicicleta, un árbol) pero muchas veces no puede recordar la apariencia específica de estos objetos.

El cerebro ha aprendido a abstraer completamente la información visual, a transformarla en conceptos visuales de alto nivel mientras filtra por completo los detalles visuales irrelevantes, haciendo que sea tremendamente difícil recordar cómo se ven exactamente las cosas a nuestro alrededor.
